In [1]:
import numpy as np
import scipy.sparse as sp

In [88]:
cover_matrix = np.array([[ 0.,  0.,   1],
       [ 1,  0.,  0.],
       [ 0.,  1,  0.]])

In [89]:
A = sp.csr_matrix(cover_matrix)

In [90]:
sets = {0:{'weight': 4},1:{'weight': 2},2:{'weight': 1}}

In [91]:
universe = [0, 1, 2]

In [2]:
class ILPSetSystem:        
    
    def setUniverse(self, U):        
        self.U = U
    
    def setSystem(self, S):        
        self.S = S
    
    def setIncMatrix(self, M):        
        self.M = M
    
    def setSystemVars(self, variables):        
        self.system_variables = variables            
    
    def setUniverseVars(self, variables):        
        self.universe_variables = variables

In [3]:
SetCover = ILPSetSystem()

In [94]:
SetCover.setSystem(sets)

In [95]:
SetCover.setIncMatrix(A)

In [96]:
SetCover.setUniverse(universe )

In [103]:
from gurobipy import *

def createModel(SetCover):
    """ Greate an ILP for the weighted set cover problem
    
        Arguments:
            SetCover     -- a weighted ILPSetSystem

        Returns:
            Gurobi model for weighted SetCover
    """
    
    # Create model
    m = Model("setsystemilp_min_set_cover")  
    
    # Add variables
    len_x = len(SetCover.S)
    len_b = len(SetCover.U)
    x = m.addMVar(shape=len_x, vtype=GRB.BINARY, name="x")
    SetCover.setSystemVars( x)
    m.update()
    
    # Add  vector b for the right-hand side
    b = np.ones((len_b,), dtype=int)
    
    # set weight vector 
    obj = np.array([val['weight'] for _set,val in SetCover.S.items()])
    
    # Add constraints
    m.addConstr(A @ x >= b, name="c")
    
    # set optimisation objective: minimize cardinality of the set cover  
    m.setObjective(obj @ x, GRB.MINIMIZE)
    
    return m

In [104]:
m = createModel(SetCover)

In [105]:
m.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 3 rows, 3 columns and 3 nonzeros
Model fingerprint: 0xcdeb162b
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 7.0000000
Presolve removed 3 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: 7 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.000000000000e+00, best bound 7.000000000000e+00, gap 0.0000%


In [106]:
def extractSolution(SetCover, model):
    """ Get a list of sets comprising a set cover
    
        Arguments:
            SetCover     -- a weighted ILPSetCover
            model        -- a solved Gurobi model for weighted SetCover
            
        Returns:
            a list of sets comprising a set cover
    """
    iterate = list(range (  len(SetCover.S) ) )
    set_cover = [list(SetCover.S.keys())[i] for i in iterate if SetCover.system_variables.X[i] > 0.5 ]
    
    return set_cover

In [107]:
set_cover = extractSolution(SetCover, m)

In [108]:
set_cover 

[0, 1, 2]